In [21]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="get_location")

location = "Morongo Valley"
location = geolocator.geocode(location)
latitude = location.latitude
longitude = location.longitude

print("Latitude:", latitude)
print("Longitude:", longitude)

Latitude: 34.0537585
Longitude: -116.59672363338927


In [22]:
def get_coordinate(query: str):
    """
    Get the latitude and longitude from an input query string
    """
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="get_location")
 
    location = geolocator.geocode(query)
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None

def partition_by_caps(input):
    import re
    parts = re.findall('[A-Z][^A-Z]*', input)
    return parts

In [23]:
import pandas as pd
df = pd.read_csv('communities.csv', index_col=None, header=None)
types = ['city', 'township', 'town', 'borough', 'village', 'division', 'district', 'Valle']
df[df[3].str[-4:]=='alle'][3]

1655    TwentyninePalms-MorongoValle
Name: 3, dtype: object

In [24]:
sep = '\s*,\s*'

# Read CSV file using custom separator
state_code_df = pd.read_csv('us-state-code.csv', sep=sep, engine='python')
def get_state_name(code:int):
    return state_code_df.query('st=='+str(code))['stname'].to_string(index=False)

get_state_name(42)

'Pennsylvania'

In [25]:
df['state_name'] = df[0].apply(get_state_name)

In [26]:
city_lst = df[3].apply(partition_by_caps).to_list()

In [27]:
type_list = []
for city in city_lst:
    for t in types:
        if t in city[-1]:
            city[-1] = city[-1].removesuffix(t)
            type_list.append(t)
            removed = True
            break
        


    
df['type'] = type_list
df['city'] = city_lst

In [28]:
query_strs = []

for i in range(len(df['type'])):
    query_msg = ''
    for name in df['city'][i]:
        query_msg += name + ' '
    query_msg = query_msg[:-1]
    query_msg += ', '+df['state_name'][i]
    
    query_strs.append(query_msg)

In [29]:
# # this will take ~ 20 min
# latitudes = []
# longitudes = []
# for i in range(len(df['type'])):
#     try:
#         coor = get_coordinate(query_strs[i])
#         if coor is not None:
#             latitudes.append(coor[0])
#             longitudes.append(coor[1])
#         else:
#             latitudes.append(None)
#             longitudes.append(None)
#     except:
#         latitudes.append(None)
#         longitudes.append(None)

In [30]:
# pd.DataFrame({'latitude':latitudes,'longitude':longitudes}).to_csv('coordinates.csv')

In [31]:
# pd.read_csv('coordinates.csv')[['latitude', 'longitude']].to_csv('coordinates.csv')

In [32]:
coor = pd.read_csv('coordinates.csv')

In [33]:
coor.isna().sum()

Unnamed: 0     0
latitude      12
longitude     12
dtype: int64

In [34]:
df['latitude'] = coor['latitude']
df['longitude'] = coor['longitude']

In [35]:

df = df.rename(columns={127: 'violent_crime_rate',
                        5:'population'})

In [36]:
cities = []
for i in range(len(df['city'])):
    city_str = ''
    for name in df['city'][i]:
        city_str += name + ' '
    city_str = city_str[:-1]
    
    cities.append(city_str)
df['community'] = cities

In [37]:
df

,0,1,2,3,4,population,6,7,8,9,...,124,125,126,violent_crime_rate,state_name,type,city,latitude,longitude,community
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.5,0.32,0.14,0.20,Colorado,city,[Lakewood],39.708574,-105.084669,Lakewood
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,?,0.00,?,0.67,Washington,city,[Tukwila],47.462736,-122.255916,Tukwila
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,?,0.00,?,0.43,Maryland,town,[Aberdeen],39.512035,-76.164329,Aberdeen
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,?,0.00,?,0.12,New Jersey,township,[Willingboro],40.027889,-74.869054,Willingboro
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,?,0.00,?,0.03,Pennsylvania,township,[Bethlehem],40.617891,-75.378652,Bethlehem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,12,?,?,TempleTerracecity,10,0.01,0.40,0.10,0.87,0.12,...,?,0.00,?,0.09,Florida,city,"[Temple, Terrace]",28.035296,-82.389260,Temple Terrace
1990,6,?,?,Seasidecity,10,0.05,0.96,0.46,0.28,0.83,...,?,0.00,?,0.45,California,city,[Seaside],36.625930,-121.816938,Seaside
1991,9,9,80070,Waterburytown,10,0.16,0.37,0.25,0.69,0.04,...,0,0.91,0.28,0.23,Connecticut,town,[Waterbury],41.553809,-73.043836,Waterbury
1992,25,17,72600,Walthamcity,10,0.08,0.51,0.06,0.87,0.22,...,0,0.22,0.18,0.19,Massachusetts,city,[Waltham],42.375640,-71.235800,Waltham


In [38]:
df.to_csv('processed_communities.csv')